In [1]:
%load_ext autoreload
%autoreload 2
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from io import StringIO 
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score

from cancellation_model import CancellationModel, load_training_data, load_prediction_data


In [2]:
train = load_training_data()
test = load_prediction_data()
train.reset_index(drop = True, inplace = True)
test.reset_index(drop = True, inplace = True)

/home/vkocheganov/virtualenvironment/nn_course/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3347: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='binary' #Binary target feature
params['metric']='binary_logloss' #metric for binary classification
params['max_depth']=10
model = CancellationModel({"model_type": "lgb", "train_params" : params, "epocs" : 100})
model.train(train)


country_code
provider_code_encoded


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 344679, number of negative: 1094019
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022068 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 707
[LightGBM] [Info] Number of data points in the train set: 1438698, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.239577 -> initscore=-1.155000
[LightGBM] [Info] Start training from score -1.155000


In [4]:
X_test = pd.DataFrame()
X_test['pred'], X_test['cancelled'] = model.predict(test)

TypeError: __preprocess() got an unexpected keyword argument 'inplace'

In [5]:
X_test.pred.sum()/X_test.cancelled.sum()


AttributeError: 'DataFrame' object has no attribute 'pred'

In [6]:

print(X_test['pred'])
model_saved = "model.txt"
model.dump(model_saved)
loaded_model = CancellationModel.load(model_saved)
X_test['pred'], X_test['cancelled'] = loaded_model.predict(test)
print(X_test['pred'])


KeyError: 'pred'

In [7]:
model.score(X_test['cancelled'], X_test['pred'])

KeyError: 'cancelled'

In [8]:
X_test['cancelled']

KeyError: 'cancelled'

In [9]:
X_test['pred']

KeyError: 'pred'